In [1]:
#from cityscapes_dataset import CityscapesDataset
from nyuv2_dataset import NYUv2Dataset
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from densenet import DenseNet

from trainer import Trainer
from utils import visualize_results_multitask, count_params, AngleDistance, MeanAbsoluteRelativeError

2025-02-05 23:44:39.688530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-05 23:44:39.727594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-05 23:44:40.388800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
prova_in = torch.randn(3)
prova_out = torch.randn(3)
tanh = torch.nn.Tanh()
out = tanh(prova_in)
prova_out = tanh(prova_out)
stat = AngleDistance()
#stat.update(out, prova_out)
stat(out, prova_out)

Preds: tensor([ 84.9453, 120.7298, 119.1726])
Targets: tensor([123.4151, 157.6932, 134.8378])
Preds: tensor([ 84.9453, 120.7298, 119.1726])
Targets: tensor([123.4151, 157.6932, 134.8378])


(tensor(10.1220), tensor(12.3211), tensor([0.0000, 0.3333, 0.3333]))

In [ ]:
x = torch.randn(2, 3, 224, 224)
y = torch.randn(2, 3, 224, 224)
B, C, H, W = x.shape
x_flat = x.view(B, 1, C*H*W)
y_flat = y.view(B, C*H*W, 1)
pq = H*W
res = -(1/pq)*torch.matmul(x_flat, y_flat).squeeze(1)
res = torch.mean(res)
print(res)

In [2]:
LABELS = 7
BATCH_SIZE = 8

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
nyuv2 = NYUv2Dataset(root="../nyuv2_preprocessed")
train_dl = DataLoader(nyuv2, batch_size=1, shuffle=True)
for image, label, depth, normals in train_dl:
    print(f'Image: {image.shape}, Label: {label.shape}, Depth: {depth.shape}, Normals: {normals.shape}')
    print(f'Image: {image.max()}, {image.min()}') 
    print(f'Label: {label.max()}, {label.min()}')
    print(f'Depth: {depth.max()}, {depth.min()}')
    print(f'Normals: {normals.max()}, {normals.min()}')
    break
ax, fig = plt.subplots(4, figsize=(10, 10))
fig[0].imshow(image[0].permute(1, 2, 0))
fig[1].imshow(label[0])
fig[2].imshow(depth[0])
fig[3].imshow(normals[0].permute(1, 2, 0))
plt.show()

In [ ]:
cityscapes_train = CityscapesDataset(root="../cityscapes_preprocessed", split="train",labels=LABELS)
train_dl = DataLoader(cityscapes_train, batch_size=BATCH_SIZE, shuffle=False)
for image, label, depth in train_dl:
    print(f'Image: {image.shape}, Label: {label.shape}, Depth: {depth.shape}')
    print(f'Image: {image.max()}, {image.min()}') 
    print(f'Label: {label.max()}, {label.min()}')
    print(f'Depth: {depth.max()}, {depth.min()}')
    break
ax, fig = plt.subplots(3, figsize=(10, 10))
fig[0].imshow(image[0].permute(1, 2, 0))
fig[1].imshow(label[0])
fig[2].imshow(depth[0])
plt.show()

cityscapes_val = CityscapesDataset(root="../cityscapes_preprocessed", split="val", labels=LABELS)
val_dl = DataLoader(cityscapes_val, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = DenseNet(classes=LABELS)
print(f"{model.name} has {count_params(model)} parameters")
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(model, opt)
trainer.train(train_dl, val_dl, epochs=2, save=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tests = 10
path = 'models/mtan/mtan_train10.pth'
model = MTAN(classes=LABELS)
model.load_state_dict(torch.load(path, weights_only=True))
model.to(device)
for i, (img, img_seg, img_depth) in enumerate(val_dl):
    visualize_results_multitask(model, img[0], img_seg[0], img_depth[0], device)
    if i == tests:
        break